In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
### STEP 1 -> 🧩 Install Dependencies
!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba

bin/micromamba


In [ ]:
### STEP 2 -> 🧩 Create and Install Bioenv with regtools
!bin/micromamba create -y -n bioenv -c bioconda -c conda-forge \
    regtools ucsc-bedtogenepred ucsc-genepredtogtf bedtools

bioconda/linux-64                                           Using cache
bioconda/noarch                                             Using cache
conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache


Transaction

  Prefix: /root/.local/share/mamba/envs/bioenv

  Updating specs:

   - regtools
   - ucsc-bedtogenepred
   - ucsc-genepredtogtf
   - bedtools


  Package                 Version  Build          Channel           Size
──────────────────────────────────────────────────────────────────────────
  Install:
──────────────────────────────────────────────────────────────────────────

  + _libgcc_mutex             0.1  conda_forge    conda-forge     Cached
  + _openmp_mutex             4.5  2_gnu          conda-forge     Cached
  + bedtools               2.31.1  h13024bc_3     bioconda        Cached
  + bzip2                   1.0.8  h4bc722e_7     conda-forge     Cached
  + ca-certificates     2

In [ ]:
### STEP 3 -> 🧩 Add the Environment to Colab’s PATH
!regtools junctions extract \
  -a 8 -m 50 -M 500000 \
  /mnt/data/elizabethae_aligned_sorted.bam \
  -o /content/elizabethae_splice_junctions.bed

/bin/bash: line 1: regtools: command not found


In [ ]:
!apt-get update
!apt-get install -y samtools

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,152 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,269 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,574 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu

In [ ]:
### -> Check for file in drive.
!ls "/content/drive/MyDrive/2025_SnrProj_Dir/Data/Elizabethae_Assembly_Trinity.fasta"

/content/drive/MyDrive/2025_SnrProj_Dir/Data/Elizabethae_Assembly_Trinity.fasta


In [ ]:
# Define shell variables for clarity
transcriptome = "/content/drive/MyDrive/2025_SnrProj_Dir/Data/Elizabethae_Assembly_Trinity.fasta"
genome = "/content/drive/MyDrive/2025_SnrProj_Dir/Genome/genome.fasta"
output_dir = "/content/drive/MyDrive/2025_SnrProj_Dir/Transcriptome_Alignments/Elizabethae"

In [ ]:
### -> 🧬 Step 2: Align Transcriptome to Genome
# Run shell commands with proper syntax in Colab
!mkdir -p /content/drive/MyDrive/2025_SnrProj_Dir/Transcriptome_Alignments/Elizabethae

!minimap2 -ax splice -uf -k14 \
  /content/drive/MyDrive/2025_SnrProj_Dir/Data/assembly.fasta \
  /content/drive/MyDrive/2025_SnrProj_Dir/Data/Elizabethae_Assembly_Trinity.fasta \
  > /content/drive/MyDrive/2025_SnrProj_Dir/Transcriptome_Alignments/Elizabethae/elizabethae_aligned.sam

!samtools view -bS /content/drive/MyDrive/2025_SnrProj_Dir/Transcriptome_Alignments/Elizabethae/elizabethae_aligned.sam \
  | samtools sort -o /content/drive/MyDrive/2025_SnrProj_Dir/Transcriptome_Alignments/Elizabethae/elizabethae_aligned_sorted.bam

!samtools index /content/drive/MyDrive/2025_SnrProj_Dir/Transcriptome_Alignments/Elizabethae/elizabethae_aligned_sorted.bam

[M::mm_idx_gen::54.048*1.29] collected minimizers
[M::mm_idx_gen::74.261*1.36] sorted minimizers
[M::main::74.273*1.36] loaded/built the index for 56291 target sequence(s)
[M::mm_mapopt_update::75.304*1.36] mid_occ = 2349
[M::mm_idx_stat] kmer size: 14; skip: 5; is_hpc: 0; #seq: 56291
[M::mm_idx_stat::75.966*1.35] distinct minimizers: 35401826 (43.52% are singletons); average occurrences: 7.073; average spacing: 2.940; total length: 736121066
[M::worker_pipeline::532.402*1.61] mapped 114949 sequences
[M::main] Version: 2.24-r1122
[M::main] CMD: minimap2 -ax splice -uf -k14 /content/drive/MyDrive/2025_SnrProj_Dir/Data/assembly.fasta /content/drive/MyDrive/2025_SnrProj_Dir/Data/Elizabethae_Assembly_Trinity.fasta
[M::main] Real time: 532.532 sec; CPU: 859.331 sec; Peak RSS: 6.078 GB


In [ ]:
%cd /content/Augustus/auxprogs/bam2hints

/content/Augustus/auxprogs/bam2hints


In [ ]:
makefile_content = """
BAMTOOLS_ROOT = /content/bamtools
BAMTOOLS_INCLUDE_DIR = $(BAMTOOLS_ROOT)/src
BAMTOOLS_LIB_DIR = $(BAMTOOLS_ROOT)/build/src

INCLUDES = -I$(BAMTOOLS_INCLUDE_DIR)
LIBRARIES = -L$(BAMTOOLS_LIB_DIR) -lbamtools

CXX = g++
CXXFLAGS = -O2 -Wall -Wno-deprecated $(INCLUDES)

all: bam2hints

bam2hints: bam2hints.cc
\t$(CXX) $(CXXFLAGS) -o bam2hints bam2hints.cc $(LIBRARIES)

clean:
\trm -f bam2hints
"""

with open("/content/Augustus/auxprogs/bam2hints/Makefile", "w") as f:
    f.write(makefile_content)

In [ ]:
%cd /content/Augustus/auxprogs/bam2hints
!make

/content/Augustus/auxprogs/bam2hints
g++ -O2 -Wall -Wno-deprecated -I/content/bamtools/src -o bam2hints bam2hints.cc -L/content/bamtools/build/src -lbamtools
In file included from /content/bamtools/src/api/BamAlignment.h:18,
                 from /content/bamtools/src/api/BamReader.h:14,
                 from bam2hints.cc:16:
/content/bamtools/src/api/BamAux.h:42:19: error: variable ‘BamTools::API_EXPORT BamTools::CigarOp’ has initializer but incomplete type
   42 | struct API_EXPORT CigarOp
      |                   ^~~~~~~
/content/bamtools/src/api/BamAux.h:45:5: error: expected primary-expression before ‘char’
   45 |     char Type;        //!< CIGAR operation type (MIDNSHPX=)
      |     ^~~~
/content/bamtools/src/api/BamAux.h:45:5: error: expected ‘}’ before ‘char’
/content/bamtools/src/api/BamAux.h:43:1: note: to match this ‘{’
   43 | {
      | ^
/content/bamtools/src/api/BamAux.h:45:5: error: expected ‘,’ or ‘;’ before ‘char’
   45 |     char Type;        //!< CIGAR operation t

In [ ]:
!ls -lh /content/Augustus/auxprogs/bam2hints

total 60K
drwxr-xr-x 13 root root 4.0K Jul 29 22:57 Augustus
-rw-r--r--  1 root root  38K Jul 29 23:03 bam2hints.cc
drwxr-xr-x  9 root root 4.0K Jul 29 22:15 bamtools
drwxr-xr-x  4 root root 4.0K Jul 29 22:59 build
-rw-r--r--  1 root root  393 Jul 30 20:57 Makefile
-rw-r--r--  1 root root  754 Jul 29 22:13 README
